In [1]:
import re
import cirpy
import openpyxl

import pandas as pd
import numpy as np 

from tqdm import tqdm

pd.set_option('mode.chained_assignment', None)

In [2]:
df_tmp = pd.read_excel('tg471_raw.xlsx', header = 0)

In [3]:
df_tmp 

Chemical        CasRN  \
0                         chloromethane; methyl chloride      74-87-3   
1                                          Methyloxirane      75-56-9   
2      2-benzyl-2-dimethylamino-4'-morpholinobutyroph...  119313-12-1   
3      2-benzyl-2-dimethylamino-4'-morpholinobutyroph...  119313-12-1   
4                                   Diisobutyl phthalate      84-69-5   
...                                                  ...          ...   
22396                                 Butyl methacrylate      97-88-1   
22397                                     Cyclopentanone     120-92-3   
22398                                     Cyclopentanone     120-92-3   
22399                                           Methanol      67-56-1   
22400                                           Methanol      67-56-1   

       Route of administration  \
0                          NaN   
1                          NaN   
2                          NaN   
3                          NaN   
4                          NaN   
...                        ...   
22396                      NaN   
22397                      NaN   
22398                      NaN   
22399                      NaN   
22400                      NaN   

                                        Species / strain Metabolic activation  \
0      S. typhimurium TA 1535, TA 1537, TA 98 and TA 100     with and without   
1                                   S. typhimurium TA 98                 with   
2                              other: as specified above                 with   
3                              other: as specified above              without   
4      other: TA 1535, TA 100, TA 1537, TA 98 and E. ...     with and without   
...                                                  ...                  ...   
22396  other: Salmonella typhimurium TA100, TA1535, T...     with and without   
22397  S. typhimurium TA 1535, TA 1537, TA 98, TA 100...     with and without   
22398  S. typhimurium TA 1535, TA 1537, TA 98, TA 100...     with and without   
22399      other: Aspergillus nidulans diploid strain P1              without   
22400      other: Aspergillus nidulans diploid strain P1              without   

      Genotoxicity Cytotoxicity / choice of top concentrations  \
0         positive                               not specified   
1         positive                               not specified   
2         negative                                cytotoxicity   
3         negative                                cytotoxicity   
4         negative                                cytotoxicity   
...            ...                                         ...   
22396     negative                                cytotoxicity   
22397     negative                             no cytotoxicity   
22398     negative                             no cytotoxicity   
22399     positive                                cytotoxicity   
22400     positive                                cytotoxicity   

                                                 Remarks  \
0                                                    NaN   
1                                                    NaN   
2                                      (> 5000 ug/plate)   
3                                       (>5000 ug/plate)   
4      observed occasionally from about 500 µg/plate ...   
...                                                  ...   
22396  >= 156 µg/plate in the 5 strains without S9. >...   
22397                                                NaN   
22398                                                NaN   
22399  7.0 % (lowest concentration to arrest conidial...   
22400  7.0 % (lowest concentration to arrest conidial...   

      Vehicle controls validity Untreated negative controls validity  \
0                           NaN                                  NaN   
1                           NaN                                  NaN   
2                           NaN                            

유전 독성  : Genotoxicity binary classification

In [4]:
df_tmp.Genotoxicity.isnull().sum()

133

In [5]:
def check_endpoint(string):
    try:
        endpoint = re.findall('positive|negative', string)
        return endpoint[0]
    except:
        return np.nan

In [6]:
# 하나의 화합물이 여러 개의 결과를 갖을 때, 보수적으로 진행하기 위해 하나라도 positive 결과를 갖으면 Genotoxicity를 postivie로 지정
def extract_endpoint(casrn):
    length = len(geno_tmp.Genotoxicity[geno_tmp.CasRN == casrn].unique())
    
    if length == 1:
        return geno_tmp.Genotoxicity[geno_tmp.CasRN == casrn].unique()[0]
    
    elif length > 1:
        # count = geno_tmp.Genotoxicity[geno_tmp.CasRN == casrn].value_counts()
        return 'positive'

In [7]:
geno_tmp = df_tmp[['Chemical', 'CasRN', 'Genotoxicity']]
geno_tmp['Genotoxicity'] = geno_tmp.Genotoxicity.map(lambda x: check_endpoint(str(x)))
geno_tmp = geno_tmp[geno_tmp.Genotoxicity.notna()].reset_index(drop = True)

geno = geno_tmp[['Chemical', 'CasRN']].drop_duplicates(['CasRN']).reset_index(drop = True)
geno['Genotoxicity'] = geno.CasRN.map(lambda x: extract_endpoint(x))

In [ ]:
# generate SMILES
tqdm.pandas()
geno['SMILES'] = geno.CasRN.progress_apply(lambda x: cirpy.resolve(x, 'smiles'))
geno.SMILES.isna().sum()
geno = geno[geno.SMILES.notna()].reset_index(drop = True)

  8%|███▏                                  | 467/5500 [09:27<1:49:07,  1.30s/it]

In [ ]:
geno.to_excel('tg471.xlsx', index = False, header = True)